# PyTorch NN Workshop
### Introduction
Welcome to the PSIML's PyTorch Neural Networks Workshop. 
### Goal
In this workshop you will build a deep learning pipeline using PyTorch. You will use datasets and dataloaders to manipulate the data. You will learn how to make new models or change existing ones. You will make a 'training loop' to train, validate and test your model. 
### Key Ingredients
* Data
* Models
    * Logistic Regression as a single layer NN
    * Multi-layer NN
* Training and Testing

In [ ]:
# some useful imports
import numpy as np
import torch
import torchvision
import pandas as pd
import PIL
import matplotlib.pyplot as plt

## Data
You have already learned how to make PyTorch datasets and dataloaders in previous workshop, so this time we will use some pre-made ones to speed things up.

Head out to [torchvision/datasets](https://pytorch.org/docs/stable/torchvision/datasets.html) and check out the [Fashion-MNIST](https://pytorch.org/docs/stable/torchvision/datasets.html#fashion-mnist) dataset.

This is a 'map style' dataset, therefore, `dataset[idx]` will return a sample at that index in form of a tuple (image, label) similar to the dataset example you saw in the PyTorch lecture.

You will notice that only train and test parts of the dataset are available. Therefore, we need to split the training part into train and validation. For this purpose it is useful to check the [SubsetRandomSampler](https://pytorch.org/docs/stable/data.html#torch.utils.data.SubsetRandomSampler). Then, we will provide the train and valid split indices to create the corresponding samplers that we will pass to the train and valid dataloaders.

Note that if you wish to add some augmentations to your training dataset, you will need to create separate datasets. In the cell below, we create only the dummy transformations to convert our data into tensors. If you wish to play with data augmentations check [torchvision.transforms](https://pytorch.org/docs/stable/torchvision/transforms.html), however, it is optional and you can complete the workshop without any augmentations.

In [ ]:
# data preprocessing and augmentations
# you can leave these as is for now (ToTensor just converts a PIL image to torch.tensor)
train_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),])  # define your train transforms = augmentation + normalization
valid_test_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),])  # define your validation/test transforms = usually just normalization

In [ ]:
# get some datasets
train_dataset = torchvision.datasets.FashionMNIST('./data', train=True, transform=train_transforms, download=True)
valid_dataset = torchvision.datasets.FashionMNIST('./data', train=True, transform=valid_test_transforms, download=True)
    # note that we make the valid dataset using same data as for the train, but with different transformations.
    # we will later split the indices to form the actual training and validation datasets
test_dataset = torchvision.datasets.FashionMNIST('./data', train=False, transform=valid_test_transforms, download=True)

# get the labels of the dataset
label_descriptions = {i:train_dataset.classes[i] for i in range(len(train_dataset.classes))}
for L in label_descriptions:
    print(f'{L}: {label_descriptions[L]}')

In [ ]:
### COMPLETE THE CODE BELOW

# What are the dimensionalities of our dataset?
img_size = None  # input dimensions
num_classes = None  # number of classes in our dataset

print(f'input dim: {img_size}   output dim: {num_classes}')

In [ ]:
### COMPLETE THE CODE BELOW

# make train and valid splits
import random
random.seed(0)  # rng seed, set to 0 for reproducibility
train_dataset_indices = list(range(len(train_dataset)))

pass  # shuffle the indices before splitting (use random.shuffle)
print(train_dataset_indices[:10])

# split the train data: 80% for training and 20% for validation
train_split_indices = None  # get the training split indices
valid_split_indices = None  # get the validation split indices 
print(valid_split_indices[:10])

In [ ]:
# we can use SubsetRandomSampler to limit which indices our dataloader will use 
# (even though we are passing the same dataset to both of the dataloaders)
train_subset_sampler = torch.utils.data.SubsetRandomSampler(train_split_indices)
valid_subset_sampler = torch.utils.data.SubsetRandomSampler(valid_split_indices)

# make dataloaders
batch_size = 1 
num_workers = 0 
train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, sampler=train_subset_sampler, num_workers=0)
valid_dataloader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=batch_size, sampler=valid_subset_sampler, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
# lets visualize some data from our datasets

for dl in [train_dataloader, valid_dataloader, test_dataloader]:
    plt.figure(figsize=(16, 5))
    idl = iter(dl)  # so that we can call 'next'
    for i in range(10):
        image,label = next(idl)
        plt.subplot(1, 10, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(label_descriptions[label.item()].title())
        plt.imshow(image.squeeze(), cmap='gray')
    plt.show()

## Models

### Logistic Regression as a single layer NN
In this section we will write the simple logistic regression model using available PyTorch functionalities.

First, take a look at https://en.wikipedia.org/wiki/Cross_entropy#Cross-entropy_loss_function_and_logistic_regression.

Therefore, we can use a [Linear](https://pytorch.org/docs/stable/nn.html#torch.nn.Linear) model plus the [Cross Entropy Loss](https://pytorch.org/docs/master/generated/torch.nn.CrossEntropyLoss.html) to define the multi-class logistic regression optimization problem.

PyTorch models are based on the [torch.nn.Module](https://pytorch.org/docs/master/generated/torch.nn.Module.html) class. It defines the model components ('\_\_init\_\_') and the operations applied to the input in terms of those components to get the output ('forward').

For example:
```Python
import torch.nn as nn
import torch.nn.functional as F
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer1 = torch.nn.Linear(in_features=100, out_features=10, bias=True)
        self.layer2 = torch.nn.Linear(in_features=10, out_features=1, bias=True)

    def forward(self, x):
        x = F.sigmoid(self.layer1(x))
        x = self.layer2(x)
        return x
```

In [ ]:
### COMPLETE THE CODE BELOW
# define a model class with a single linear layer with bias
# we do not need the softmax activation function since  
# the CELoss that we'll use later performs the softmax for us
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(0)  # rng for reproducibility

class LinearModel(torch.nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        self.layer1 = None  # make a linear layer with bias; 
                            # input is the flattened image
                            # and we need 1 output per class
    def forward(self, x):
        pass

with torch.no_grad():
    image = torch.ones(28*28)
    model = LinearModel()
    print(model(image))

### Training loop
We will train a logistic regression model on Fashion-MNIST data, applying all the things we have learned so far. We will provide you with the code (similar to what you have seen in the logistic regression workshop) that you can imediately play with.

We will use the ready-made PyTorch dataset for Fashion-MNIST from the torchvision package.

Try playing with the following parameters and see if you can get better results or speed-up:
- learning_rate
- device (NOTE: you will need a gpu with cuda in order to play with this parameter)
- batch_size
- num_epochs
- num_workers

In [ ]:
# Define the meta parameters of training

# create the model
model = LinearModel()  # our linear model

# create the SGD optimizer 
learning_rate = 0.01
optimizer = optim.SGD(model.parameters(), lr=learning_rate)  # we will use stochastic gradient descent
    
# create the CrossEntropy Loss function
loss_func = nn.CrossEntropyLoss()  # we will use cross entropy loss

# make dataloaders
num_epochs = 2
batch_size = 1000 
num_workers = 0
train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, 
                                               sampler=train_subset_sampler, num_workers=num_workers)
valid_dataloader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=batch_size, 
                                               sampler=valid_subset_sampler, num_workers=num_workers)
test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, 
                                              shuffle=False, num_workers=num_workers)



In [ ]:
### COMPLETE THE CODE BELOW
import time

dtype = torch.float
device = torch.device("cpu")  # change "cpu" to "cuda" to run on your GPU

# Training Loop
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()  # sets the model into training mode.
                   # (important if you use batch_norm, droupout and similar layers 
                   # that behave differently during training and during evaluation)
            
    for i, (x,y) in enumerate(train_dataloader):
        x = None  # flatten 'x' and move it to device
        y = None  # move 'y' to device

        # Use autograd to compute the backward pass. This call will compute the
        # gradient of loss with respect to all Tensors with requires_grad=True.
        # After this call model parameters '.grad' will be the Tensors holding the gradient
        # of the loss with respect to w. Remember to zero out the gradients before each pass.
        
        pass  # zero the gradient buffers of the optimizer using .zero_grad()
        output = None  # compute the model output (model() is equivalent to model.forward())
        loss = None  # compute the loss between the model output and the labels using the loss function
        pass  # compute the gradients of the loss using .backward()
        pass  # Update the model weights via SGD (optimizer) using .step() 
        
        # Note that you can also manually update weights 
        # for f in model.parameters():
        #     f.data.sub_(f.grad.data * learning_rate)
        
        
        # print the loss
        if (i+1) % (6000//batch_size) == 0:
            print(f'epoch: {epoch}   iter: {i+1}   batch_loss: {loss}')       
    
    # Calculate validation accuracy
    correct = 0
    total = 0
    
    model.eval()  # sets the model into evaluation mode 
                  # (important if you use batch_norm, droupout and similar layers 
                  # that behave differently during training and during evaluation)
            
    with torch.no_grad(): # we don't need to track gradients during evaluation
        for x, y in valid_dataloader:
            x = None  # flatten 'x' and move it to device
            y = None  # move 'y' to device
            output = None  # compute the model output (model() is equivalent to model.forward())
            
            y_pred = None  # convert logits (model outputs) to class probabilities 
                           # (use torch.softmax or log_softmax)
            
            _, predicted = torch.max(y_pred, 1)  # find the most probable class (use torch.max)
            
            total+= y.size(0)
            correct+= (predicted == y).sum()
        validation_accuracy = 100 * float(correct)/total
        print(f'epoch: {epoch}   validation accuracy: {validation_accuracy}%' )
        
        
    # print epoch runtime
    end_time = time.time()
    print(f'--- Epoch completed in {end_time-start_time} seconds --- \n')

### Multi-layer NN

Now that you have familiarized yourself with the basic training pipeline, we will do some experiments with the model architecture

model (neural network) 
- you can see the Logistic Regression model as a single-layer neural network, now try adding more layers ([torch.nn.Linear](https://pytorch.org/docs/stable/nn.html#torch.nn.Linear))
- try different activation functions between the layers or at the output ([torch.nn.functional](https://pytorch.org/docs/stable/nn.functional.html#non-linear-activation-functions))
- how does changing the final activation affect the loss function?
    
As above, try playing with the following parameters and see if you can get better results or speed-up:
- learning_rate (you can try different [optimizers](https://pytorch.org/docs/stable/optim.html) as well)
- device (NOTE: you will need a gpu with cuda in order to play with this parameter)
- batch_size
- num_epochs
- num_workers

In [ ]:
### COMPLETE THE CODE BELOW
# define your model class

class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        pass
    def forward(self, x):
        pass
    

In [ ]:
### COMPLETE THE CODE BELOW
# Go ahead and make your own training pipeline following the code outline defined above.

# create the model
model = None  # instance your new model

# create the optimizer 
learning_rate = None
optimizer = None  # use SGD as above, or try something else, e.g. Adam
    
# create the CrossEntropy Loss function
loss_func = None


# create transforms for data preprocessing and augmentations
train_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])  # you might add some augmentations and normalization
valid_test_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])  # define your validation/test transforms = usually just normalization

# get some datasets
train_dataset = torchvision.datasets.FashionMNIST('./data', train=True, transform=train_transforms, download=True)
valid_dataset = torchvision.datasets.FashionMNIST('./data', train=True, transform=valid_test_transforms, download=True)
test_dataset = torchvision.datasets.FashionMNIST('./data', train=False, transform=valid_test_transforms, download=True)

# make dataloaders
num_epochs = None
batch_size = None
num_workers = 0

train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, sampler=train_subset_sampler, num_workers=num_workers)
valid_dataloader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=batch_size, sampler=valid_subset_sampler, num_workers=num_workers)
test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [ ]:
### COMPLETE THE CODE BELOW
# Copy the code from TL2 cell and train your best model.

pass

## The Competition
### Single fully connected model
Your task is to use the framework you wrote above to train the best possible model with the following constraints:
- single model only, no ensembles
- no convolutional or pooling layers (you can use dropout and/or batch norm)
- any activation functions
- any loss, any optimizer
- only 10 epochs allowed for training

Note: Remember that you can add augmentations to your training dataset and normalization to all datasets :)

Once you are satisfied with the achieved validation accuracy, continue below to compute the test accuracy.
Some scores on the FashionMNIST are provided below, however, most of these were trained for hundreds of epochs, and with a lot of augmentations so don't get discouraged if you dont beat all of them :)
- Human performance: 83.5%
- Logistic regression: 83.9%
- MLP (two layers): 87.4%
- MLP (four layers): 88.3%
- ConvNet (2 layers only): 91.9%
- GoogLeNet: 93.7
- WideResNet with heay augmentations: 96.3% (current State-of-the-Art)


In [ ]:
### COMPLETE THE CODE BELOW
# Calculate test accuracy similarly to how you calculated the validation set accuracy

pass

test_accuracy = None

print(f'Final test accuracy: {test_accuracy}%' )